# Preprocessing

In [ ]:
import pandas as pd
import re
from tqdm import tqdm
import os
import json

In [ ]:
def cleaning(content):
    content = content.replace(".\n\n",". \n\n")
    content = content.replace(".",". ")
    punctuation = u'''!()-[]|{};:'"\,<>/?@#$%^&*_~—“””•■\u2019'''  # punctuation
    content = re.sub('\s+', ' ', content).strip()
    
    for idx, i in enumerate(content):
        if i in punctuation:
            content = content.replace(i, "")
    

    content_tmp = (content + '.')[:-1] # copy string to new pointer
    done = 0
    for idx, i in enumerate(content_tmp): 
        try: # 
            if i == '.' and content_tmp[idx+2].isdigit() and content_tmp[idx-1].isdigit():
                idx_origin = idx - done
                content = content[:idx_origin+1] + content[idx_origin+2:]
                done += 1
        except Exception as e :
            continue

    
    content = re.sub("https*\S+", " ", content)

    return content

In [ ]:
a = 'up to 4. 2.'
print(cleaning(a))

## Evaluating Extractive Summarization Techniques on News Articles (EEST)

In [ ]:
# read by default 1st sheet of an excel file
df = pd.read_excel('raw_data\EEST.xlsx')

In [ ]:
df.theme.unique()

In [ ]:
df_business = df[df.theme == 'business']
df_business = df_business[['id','human_summary','content']]

In [ ]:
content_list = list()
summary_list = list()
for index, row in tqdm(df_business.iterrows()) :
    try:
        content = cleaning(row['content'])
        #content_sentences = nltk.sent_tokenize(content)

        summary = cleaning(row['human_summary'])
        #summary_sentences = nltk.sent_tokenize(summary)

        content_list.append(content)
        summary_list.append(summary)
    except Exception as e :
        print('ERORR:', row)
        print(e)

df_business["content"] = content_list
df_business["human_summary"] = summary_list

In [ ]:
for index, row in tqdm(df_business.iterrows()) :
    if 'bundesbank' in row['content']: print(row['content'])

In [ ]:
for index, row in tqdm(df_business.iterrows()) :
    _id = row['id']
    with open(f'preprocessed_data\eest\{_id}.story', 'w', encoding="utf-8") as f:
        f.write(row['content'])

    with open(f'preprocessed_data\eest\{_id}.story', 'a', encoding="utf-8") as f:
        f.write('\n\n@highlight\n')
        #f.write('\n\n@highlight\n'.join(row['human_summary']))
        f.write(row['human_summary'])

## BBC News

In [ ]:
path_content = 'raw_data\BBC_News\content'
path_summary= 'raw_data\BBC_News\summary'
for file_name in tqdm(os.listdir(path_content)):
    with open(f'{path_content}\{file_name}') as f:
        content = f.read()
    with open(f'{path_summary}\{file_name}') as f:
        summary = f.read()
    
    content = cleaning(content)
    summary = cleaning(summary)

    # if 'bundesbank' in summary: print(summary)
    # content_sentences = nltk.sent_tokenize(content)
    # summary_sentences = nltk.sent_tokenize(summary)

    # create file
    with open(f'preprocessed_data\BBC_News_preprocessed\{file_name.split(".")[0]}.story', 'w') as f:
        f.write(content)

    with open(f'preprocessed_data\BBC_News_preprocessed\{file_name.split(".")[0]}.story', 'a') as f:
        f.write('\n\n@highlight\n')
        f.write(summary)

## Bloomberg

In [ ]:
def find_text(d,result_text = ''):
    if isinstance(d, dict):
        if 'text' in d:
            result_text += d['text']
        else:
            for k, v in d.items():
                result = find_text(v)
                if result is not None:
                    result_text +=result
    elif isinstance(d, list):
        for item in d:
            result = find_text(item)
            if result is not None:
                result_text +=result
    
    return result_text


In [ ]:
path = "..\\news_rapid_api\\full_text"
file_names = list()
content_list = list()
summary_list = list()
for file_name in tqdm(os.listdir(path)):
    summary = ''
    content_text = ''
    try:
        with open(f'{path}\{file_name}', encoding="utf8") as f:
            content =  json.loads(f.read())
        summary = '. '.join(content['abstract'])
        for component in content['components']:
            if 'parts' in component:
                for part in component['parts']:
                    content_text += find_text(part)
        content_text = cleaning(content_text)
        summary = cleaning(summary)
        # print(summary)
        # print(content_text)
        file_names.append(file_name)
        content_list.append(content_text)
        summary_list.append(summary)
    except: pass

In [ ]:
for index, value in tqdm(enumerate(file_names)) :
    with open(f'..\dataset\\finance_preprocessed\\bloom_berg\\{value}.story', 'w', encoding="utf-8") as f:
        f.write(content_list[index])
        f.write('\n\n@highlight\n')
        f.write(summary_list[index])

# Summary Dataset

In [ ]:
bbc = len(os.listdir(r'..\dataset\finance_preprocessed\BBC_News_preprocessed'))
bb = len(os.listdir(r'..\dataset\finance_preprocessed\bloom_berg'))
eest = len(os.listdir(r'..\dataset\finance_preprocessed\eest'))




print('BBC_News_preprocessed: ',bbc)
print('Bloom_berg: ',bb)
print('EEST: ',eest)
print(f'total = {bbc+bb+eest}')